In [1]:
from pymongo import MongoClient
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import re
import os

In [2]:
client = MongoClient('localhost', 27017)
db = client['finandb']
collection = db['fin_articles']

In [3]:
"""
dtobj = datetime(1992, 4, 7)
collection.insert_one({'_id': 0, 'folder': 'gdp','date':dtobj,'textfile':'ABC.txt','text':'XYZ',})
collection.delete_one({'_id':0})
"""

"\ndtobj = datetime(1992, 4, 7)\ncollection.insert_one({'_id': 0, 'folder': 'gdp','date':dtobj,'textfile':'ABC.txt','text':'XYZ',})\ncollection.delete_one({'_id':0})\n"

In [3]:
dblist = client.list_database_names()
print(dblist)

['admin', 'config', 'finandb', 'local', 'pbc', 'pbcT']


In [5]:
"""
all_documents = collection.find()
for document in all_documents:
    print(document)
"""

'\nall_documents = collection.find()\nfor document in all_documents:\n    print(document)\n'

In [5]:
def get_all_file_paths(directory):
    file_paths = []
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            full_path = os.path.join(dirpath, filename)
            file_paths.append(full_path)
    return file_paths

directory = "Macro_Economic_Indicators"  
all_files = get_all_file_paths(directory)
print(len(all_files))

81


In [100]:

months_full=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
months_short =  ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
months_all = months_full + months_short


vals = []
cntr = 0
nID = 1

for filepath in tqdm(all_files[:]):
    pathstrs = filepath.replace('\\',',').split(',')
    folder_name = pathstrs[1]
    file_name = pathstrs[2]
    #print(filepath)
    #print(folder_name,file_name)

    with open(filepath, 'r',encoding='utf-8') as f:
        tlines = f.readlines()
    #print(tlines[0])
    #print(tlines)
    prev_year = -1
    prev_month = -1
    prev_day = -1
    
    new_year = 0
    new_month = 0
    new_day = 0
    
    hr_cnt = 0
    dt_cnt = 0 
    
    prev_line_i = -1
    collector = []
    hdr_cnt = 0
    
    prev_hrs = False
    
    tlines += 'Jan ET'
    
    for j,lin in enumerate(tlines):
        linc = lin.replace('.','').replace(',','').replace('-',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')
        found_header = False
        for mi,mon in enumerate(months_all):
            if len(linc) < 45:
                hrs_check = ('hours ago' in linc) or ('min ago' in linc)
                if ((mon in linc) and (' ET' in linc)) or hrs_check :
                    
                    #print(len(linc))
                    #print('line num',j)
                    #print(linc)
                    
                    if not hrs_check:
                        month_num = mi+1
                        if month_num>12:
                            month_num -= 12
                        dt_cnt += 1
                        
                        strs = linc.split(' ')
                        #print(strs)
                        strs = [int(s) for s in strs if s.isdigit()]
                        #print(strs)
                        day = 0
                        year = 0
                        for v in strs:
                            if v>0 and v<32:
                                day = v
                                break
                                
                        for v in strs:
                            if v>1000 and v<3000:
                                year = v
                                break
                                
                        new_year = year
                        new_month = month_num
                        new_day = day
                        

                        
                    else:
                        hr_cnt += 1
                        new_year = 0
                        new_month = 0
                        new_day = 0
                        
                        
                    if hdr_cnt==0:
                        prev_year = new_year
                        prev_month = new_month
                        prev_day = new_day
                        
                        
                    #print('____')
                    found_header = True
                    hdr_cnt += 1
                    break
                    
                    
        if found_header and hdr_cnt>1:
            
            
            
            if prev_year > 0 and prev_month > 0 and prev_day > 0:
                
                articStr = '' 
                for cline in collector[1:]:
                    articStr += cline
                articStr = articStr.replace("\n\n\n",'\n').replace("\n\n",'\n').replace("\n\n",'\n')
                
                dtobj = datetime(prev_year,prev_month,prev_day)
                collection.insert_one({'_id': nID, 'folder': folder_name,'date':dtobj,'textfile':file_name,'text':articStr})
                nID += 1
                
                """
                
                print('********')
                print(folder_name)
                print(file_name)
                print('C',collector[0])
                print(prev_day,prev_month,prev_year)
                print(articStr)
                print('********')
                """
                
            prev_year = new_year 
            prev_month = new_month
            prev_day = new_day
            
            cntr += 1
            collector = []

        collector.append(lin)
            
                    
                    
    #print(hr_cnt)
    #print(dt_cnt)
    #print( (hr_cnt*100)/(hr_cnt+dt_cnt)   )
    vals.append( (hr_cnt*100)/(hr_cnt+dt_cnt)   )


100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:55<00:00,  1.45it/s]


In [101]:
nID

85209

In [ ]:
#result = collection.delete_many({})

In [4]:
all_documents = collection.find({'_id':7})
lens = []
for document in all_documents:
    print(document)

{'_id': 7, 'folder': 'cpi', 'date': datetime.datetime(2013, 4, 8, 0, 0), 'textfile': 'cpi_2013-1-1_to_2013-7-1.txt', 'text': 'A roundup of economic news from around the Web.\n--Chained CPI:Peter Orszag says chained CPI may not be a big money saver. "News that the White House will propose a new cost-of-living index in the budget it releases this week has brought joy to deficit scolds and consternation to defenders of Social Security. The measure, called the chained consumer price index, would lower the annual payment increases for Social Security beneficiaries, saving the government money as it lowers the future monthly income of retirees and disabled Americans. The change would also raise revenue over time because it would cause more taxpayers to wind up in higher marginal brackets.What neither side seems to have noticed, however, is that the difference between the chained CPI and the standard CPI has been diminishing. That means the impact of switching indexes may not be as great as m

In [214]:
#all_documents = collection.find({'_id':1})
all_documents = collection.find()
lens = []
for document in all_documents:
    #print(document)
    #lens.append( len(document['text'].split(' ')) )
    lens.append( length_function(document['text'])  )

In [233]:
np.mean(lens)

(1220.6407496948643, 903.2490845929959)

In [234]:
np.percentile(lens, 98.05)

5006.0

In [5]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

enc = tiktoken.get_encoding("cl100k_base")

def length_function(text: str) -> int:
    return len(enc.encode(text))


# The default list of split characters is [\n\n, \n, " ", ""]
# Tries to split on them in order until the chunks are small enough
# Keep paragraphs, sentences, words together as long as possible
splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=5000, 
    chunk_overlap=100,
    length_function=length_function,
)



In [3]:
from langchain.docstore.document import Document

In [75]:


all_documents = collection.find()
cnt = 0
lc_all_docs = []
N = 0
for document in all_documents:
    no_tokens = length_function(document['text']) 
    if no_tokens <= 5000:
        cnt += 1
        #splits_ar.append(len(splits))
        splits = splitter.split_text(  document['text']   )
        metad = {  'article_id': document['_id'], 'folder': document['folder'],'date':int(document['date'].strftime('%Y%m%d')),'textfile':document['textfile'] }
        #metad = {  'article_id': document['_id'], 'folder': document['folder'],'textfile':document['textfile'] }
        docis = [  Document(page_content=document['text'], metadata=metad)  ]
        lc_all_docs += docis
        
        """
        N+=1
        if N==10:
            break
        """

In [73]:
all_documents = collection.find()
token_count = 0
for document in all_documents:
    no_tokens = length_function(document['text']) 
    if no_tokens <= 5000:
        token_count += no_tokens
        
print(token_count)

81471055


In [74]:
token_count/1000

81471.055

In [76]:
len(lc_all_docs)

83544

In [79]:
81471055/len(lc_all_docs)

975.1873862874653

In [17]:
lc_all_docs

[Document(page_content='The operator of the portrait studios inside stores run by Sears and Wal-Mart stores filed for bankruptcy and will liquidate its remaining assets.\nCPI Corp., which closed its U.S. operations last month, filed for Chapter 7 on Wednesday in U.S. Bankruptcy Court in Wilmington, Del.\nThe company laid off more than 4,000 employees as a result of the studio closings, which the board of directors authorized on April 3. CPI notified Sears the next day, and by April 5 the portrait studio had stopped snapping photos.\nSears Holding Corp. terminated its agreement with CPI as of April 5, according to disclosures filed with the U.S. Securities and Exchange Commission. Toys "R" Us Inc., which also had an agreement with CPI, demanded immediate payment of $568,443 and terminated its contract with CPI. Likewise, Wal-Mart Stores Inc. ended its contract and gave CPI until April 8 to remove its property or be charged $8,000 per leased space, but CPI didn\'t remove its property in 

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embedding_function = OpenAIEmbeddings()

In [78]:
len(lc_all_docs)

83544

In [ ]:
list( range(len(lc_all_docs)     )

In [7]:
from langchain.vectorstores import Chroma

#vectordb = Chroma.from_documents(lc_all_docs, embedding_function, persist_directory="./chroma_dbf2")
#vectordb = = Chroma(persist_directory="./chroma_db2", embedding_function=embedding_function) #load

In [8]:
import time

"""

def batch_process(data_list, batch_size):
    for i in range(0, len(data_list), batch_size):
        yield data_list[i:i + batch_size]

data_list = lc_all_docs[81250:]
batch_size = 250

for batch_number, batch in enumerate(batch_process(data_list, batch_size), start=1):
    print(f"Processing batch {batch_number}, {len(batch)} elements")
    vectordb = Chroma.from_documents(batch, embedding_function, persist_directory="./chroma_dbf3")
    print('waiting for 60 sec')
    time.sleep(60)
"""

'\n\ndef batch_process(data_list, batch_size):\n    for i in range(0, len(data_list), batch_size):\n        yield data_list[i:i + batch_size]\n\ndata_list = lc_all_docs[81250:]\nbatch_size = 250\n\nfor batch_number, batch in enumerate(batch_process(data_list, batch_size), start=1):\n    print(f"Processing batch {batch_number}, {len(batch)} elements")\n    vectordb = Chroma.from_documents(batch, embedding_function, persist_directory="./chroma_dbf3")\n    print(\'waiting for 60 sec\')\n    time.sleep(60)\n'

83544

In [9]:
vectordb =  Chroma(persist_directory="./chroma_dbf3", embedding_function=embedding_function)

In [10]:
len(vectordb.get()['documents'])

83544

In [65]:
#vectordb.get(where={"folder":"cpi"  },include=['embeddings', 'documents', 'metadatas'])

{'ids': ['3e58e6fc-58cc-11ee-9469-e894f61ea51c',
  '3e58e6fd-58cc-11ee-b368-e894f61ea51c',
  '3e58e6fe-58cc-11ee-9b8f-e894f61ea51c',
  '3e58e6ff-58cc-11ee-8a55-e894f61ea51c',
  '3e58e700-58cc-11ee-afd7-e894f61ea51c',
  '3e58e701-58cc-11ee-b8f4-e894f61ea51c',
  '3e58e702-58cc-11ee-841f-e894f61ea51c',
  '3e58e703-58cc-11ee-a3d6-e894f61ea51c',
  '3e58e704-58cc-11ee-b800-e894f61ea51c',
  '3e58e705-58cc-11ee-9069-e894f61ea51c'],
 'embeddings': [[-0.020191735890295194,
   0.002677659202044496,
   0.020669889967831402,
   -0.03978236635188695,
   -0.018716291736609426,
   0.03461830902001886,
   -0.00753433210239739,
   0.003931104445717553,
   0.011837713212287463,
   -0.03885338320282316,
   0.017104231866106843,
   0.03863479805591626,
   0.01219291221336591,
   0.0027340130688161017,
   0.019262752741231407,
   0.010239313982143934,
   0.006274056072883494,
   -0.009242022974671738,
   0.0027596283993039246,
   -0.03896267577627661,
   -0.013736665156782716,
   0.014522203200172801,
   -0

In [72]:
#vectordb.get(where={"date": {"$gte": 0},"date": {"$lte": 9} },include=['embeddings', 'documents', 'metadatas'])

{'ids': [],
 'embeddings': [[-0.005956880724476562,
   0.02481703815096889,
   0.014908693437156992,
   -0.018576495685684757,
   -0.013826823588571069,
   -0.02227068509455096,
   -0.012903276702015767,
   -0.04372337473786786,
   -0.021742943750427214,
   -0.04564962906067127,
   0.017824465806990545,
   0.03699467027198389,
   -0.010086457347604477,
   -7.586282403855369e-05,
   -0.025898907068232318,
   0.014921887482987459,
   0.03638776837816733,
   0.01955281782423943,
   0.014024726825448097,
   -0.020621493626994884,
   -0.0097830054693737,
   -0.0009672509992156024,
   -0.0151989512695573,
   0.0015791011840874888,
   -0.007150895771670218,
   0.003753560519627685,
   0.025925295159893252,
   -0.0011907164193207267,
   0.009789602957950182,
   -0.0065044126716847575,
   0.005495107281198911,
   -0.023919878424752027,
   -0.032165837625177385,
   0.009063958376949411,
   -0.022574137904104234,
   -0.012177632121014996,
   0.003349508582136772,
   -0.01790362635668336,
   0.022

In [11]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [147]:
response_schemas = [
    ResponseSchema(name="Start_Year", description="Starting Year as a number"),
    ResponseSchema(name="Start_Month", description="Starting Month as a number"),
    ResponseSchema(name="Start_Day", description="Starting Day as a number"),
    ResponseSchema(name="End_Year", description="Ending Year as a number"),
    ResponseSchema(name="End_Month", description="Ending Month as a number"),
    ResponseSchema(name="End_Day", description="Ending Day as a number"),
    ResponseSchema(name="Economic_Indicator", description="Macro Economic Indicator which will be one of GDP, Inflation Rate, CPI, Interest Rate or Unemployment Rate"),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [153]:
format_instructions = output_parser.get_format_instructions()
prompt_datefinder = PromptTemplate(
    template="Find the Starting and Ending Dates of a Date Range described in the question and write it in a format similar to Start Year:2020, Start Month:4, Start Day:7, End Year:2021, End Month:5 and End Day:23. If only a single year is mentioned then date range will be from 1st January to 31st December of the same year. Also find Economic_Indicator which will be one of GDP, Inflation Rate, CPI, Interest Rate on Unemployment Rate. If any of those fields are not described in Question then write -1 for them Question is given at the end. .\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [124]:
prompt_datefinder

PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"Start_Year": string  // Starting Year as a number\n\t"Start_Month": string  // Starting Month as a number\n\t"Start_Day": string  // Starting Day as a number\n\t"End_Year": string  // Ending Year as a number\n\t"End_Month": string  // Ending Month as a number\n\t"End_Day": string  // Ending Day as a number\n\t"Economic_Indicator": string  // Macro Economic Indicator which will be one of GDP, Inflation Rate, CPI, Interest Rate or Unemployment Rate\n}\n```'}, template='Find the Starting and Ending Dates of a Date Range described in the question and write it in a format similar to Start Year:2020, Start Month:4, Start Day:7, End Year:2021, End Month:5 and End Day:23. If one year is mentioned then date range will be from start to en

In [18]:
model = OpenAI(temperature=0)

In [19]:
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.chains import ConversationalRetrievalChain

In [20]:
def getDayMonthYearFromInt(int_date):
    date_str = str(int_date)
    year = int(date_str[0:4])
    month = int(date_str[4:6])
    day = int(date_str[6:8])
    return (day,month,year)

def getIntFromDatetime(datetime_obj):
    return int(datetime_obj.strftime('%Y%m%d'))

print(getDayMonthYearFromInt(20151216))
print( getIntFromDatetime(datetime.now()) )

(16, 12, 2015)
20231003


In [21]:
def is_leap_year(year):
    """Check if a year is a leap year."""
    if year % 4 != 0:
        return False
    elif year % 100 != 0:
        return True
    elif year % 400 != 0:
        return False
    else:
        return True

def days_in_month(month, year):
    """Return the number of days in a month for a given year."""
    if month in [1, 3, 5, 7, 8, 10, 12]:  # January, March, May, July, August, October, December
        return 31
    elif month in [4, 6, 9, 11]:  # April, June, September, November
        return 30
    elif month == 2:  # February
        return 29 if is_leap_year(year) else 28
    else:
        return -1

In [102]:
def getDateFilterFromDateDict(parsed_dict):
    
    print( parsed_dict)

    yearS = int(parsed_dict['Start_Year'])
    monthS = int(parsed_dict['Start_Month'])
    dayS = int(parsed_dict['Start_Day'])
    
    yearE = int(parsed_dict['End_Year'])
    monthE = int(parsed_dict['End_Month'])
    dayE = int(parsed_dict['End_Day'])
    
    if yearE==-1:
        if yearS!=-1:
            yearE = yearS
        else:
            return {}
        
    if yearS==-1:
        if yearE!=-1:
            yearS = yearE
        else:
            return {}
    
    if monthS==-1:
        monthS=1
        
    if monthE==-1:
        monthE=12
        
    if dayS==-1:
        dayS=1
        
    if dayE==-1:
        dayE = days_in_month(monthE, yearE)
        
    econ_dict = {'GDP':'gdp', 'Inflation Rate':'inflation_rate', 'CPI':'cpi', 'Interest Rate':'interest_rate','Unemployment Rate':'unemployment_rate'}
    filter_query = {"$and":[]}
    if dayS!=-1 and monthS!=-1 and yearS!=-1 and dayE!=-1 and monthE!=-1 and yearE!=-1:
        print( {'Start_Year': yearS,'Start_Month': monthS,'Start_Day': dayS,'End_Year': yearE,'End_Month':monthE,'End_Day': dayE})
        datetimeS = datetime(yearS, monthS, dayS)
        datetimeE = datetime(yearE, monthE, dayE)
        gte_val = getIntFromDatetime(datetimeS)
        lte_val =  getIntFromDatetime(datetimeE)
        filter_query["$and"].append(  {"date": {"$gte": gte_val}}  )
        filter_query["$and"].append(  {"date": {"$lte": lte_val}}  )
        
        if parsed_dict['Economic_Indicator'] != -1:
            econfolder = econ_dict[ parsed_dict['Economic_Indicator'] ]
            filter_query["$and"].append(  {'folder': {'$eq': econfolder}}  )
        
        #if parsed_dict['Economic_Indicator'] != -1:
        #    filter_query['$eq'] = [  {'folder': econ_dict[ parsed_dict['Economic_Indicator'] ]    } ]
    else:
        if parsed_dict['Economic_Indicator'] != -1:
            econfolder = econ_dict[ parsed_dict['Economic_Indicator'] ]
            filter_query["$and"].append(  {'folder': {'$eq': econfolder}}  )
                                   
    if len(filter_query["$and"])==0:
        filter_query = {}
                                   
    return filter_query,parsed_dict
        

In [23]:
parsed_dict = {'Start_Year': 2018,
 'Start_Month': -1,
 'Start_Day': -1,
 'End_Year': 2020,
 'End_Month': 4,
 'End_Day': -1}
print(getDateFilterFromDateDict(parsed_dict))

{'Start_Year': 2018, 'Start_Month': 1, 'Start_Day': 1, 'End_Year': 2020, 'End_Month': 4, 'End_Day': 30}
{'$and': [{'date': {'$gte': 20180101}}, {'date': {'$lte': 20200430}}]}


In [40]:
"""
qa = ConversationalRetrievalChain.from_llm(
                        OpenAI(temperature=0),
                        VectorStoreRetriever(vectorstore=vectordb, search_kwargs={"filter":{"folder":"xyz"},"k":1},),
                        verbose = True,
                        return_source_documents=True)
"""

In [42]:
"""
query='The lights'
#rdocs = vectordb.similarity_search(query,k=1, where={"date": {"$gte": 20130405} }  )
rdocs = vectordb.similarity_search(query,k=1, where={"folder":"xyz"  }  )
print(len(rdocs))
"""

1


In [130]:
def getDateFilterFromQuestion(question_text):
    _input = prompt_datefinder.format_prompt(question=question_text)
    output = model(_input.to_string())
    parsed_dict = output_parser.parse(output)
    print('start')
    print(parsed_dict)
    return getDateFilterFromDateDict(parsed_dict)

In [25]:
def get_grouped_docs_dict(retrieved_docs):
    docs_folder_dict = {}
    for rdoc in retrieved_docs:
        metadct = rdoc.metadata
        folder = metadct['folder']
        if folder not in docs_folder_dict:
            docs_folder_dict[folder] = [rdoc]
        else:
            docs_folder_dict[folder].append(rdoc)
    return docs_folder_dict

In [291]:
question_prompt = 'Analyze GDP in 2018'

In [292]:
filter_query,parsed_dict = getDateFilterFromQuestion(question_prompt)
print(filter_query)

start
{'Start_Year': '2018', 'Start_Month': '1', 'Start_Day': '1', 'End_Year': '2018', 'End_Month': '12', 'End_Day': '31', 'Economic_Indicator': 'GDP'}
{'Start_Year': '2018', 'Start_Month': '1', 'Start_Day': '1', 'End_Year': '2018', 'End_Month': '12', 'End_Day': '31', 'Economic_Indicator': 'GDP'}
{'Start_Year': 2018, 'Start_Month': 1, 'Start_Day': 1, 'End_Year': 2018, 'End_Month': 12, 'End_Day': 31}
{'$and': [{'date': {'$gte': 20180101}}, {'date': {'$lte': 20181231}}, {'folder': {'$eq': 'gdp'}}]}


In [293]:
parsed_dict

{'Start_Year': '2018',
 'Start_Month': '1',
 'Start_Day': '1',
 'End_Year': '2018',
 'End_Month': '12',
 'End_Day': '31',
 'Economic_Indicator': 'GDP'}

In [294]:
econ_dict = {'GDP':'gdp', 'Inflation Rate':'inflation_rate', 'CPI':'cpi', 'Interest Rate':'interest_rate','Unemployment Rate':'unemployment_rate'}
econ_dict_rev = {v: k for k, v in econ_dict.items()}

In [295]:
econ_dict_rev

{'gdp': 'GDP',
 'inflation_rate': 'Inflation Rate',
 'cpi': 'CPI',
 'interest_rate': 'Interest Rate',
 'unemployment_rate': 'Unemployment Rate'}

In [296]:
metric = parsed_dict['Economic_Indicator']
question_prompt = 'Analyze and compare {} forecast and Actual Announced {}'.format(metric,metric) +'. '+question_prompt

In [297]:
question_prompt

'Analyze and compare GDP forecast and Actual Announced GDP. Analyze GDP in 2018'

In [298]:
K=20

In [299]:
chroma_retriever = VectorStoreRetriever(vectorstore=vectordb, search_kwargs={"filter":filter_query,"k":K},)
retrieved_docs = chroma_retriever.get_relevant_documents(question_prompt)
print(len(retrieved_docs))

20


In [300]:
retrieved_docs[0].metadata

{'article_id': 13833,
 'date': 20180709,
 'folder': 'gdp',
 'textfile': 'gdp_2018-7-1_to_2019-1-1.txt'}

In [301]:
retrieved_docs[0].page_content

'If you’re looking for the most accurate view of economic growth, you won’t find it in the government’s top-line statistics. Last month the Bureau of Economic Analysis revised its estimate for the first quarter of 2018; the BEA now says gross domestic product grew 2%, annualized, down from the previous figure of 2.2%. This dramatically understates the economy’s actual performance. A better measure, factoring in statistics on incomes, shows growth steaming along at a much stronger 2.8%.\nWhy the difference? In short, because the data aren’t perfect. Unlike the unemployment rate, which comes from a (relatively) straightforward survey, GDP is not measured directly. Instead, the BEA sums up economy-wide expenditures from dozens of data sources, covering consumption, investment, government spending, net exports and more.\nLast month’s figure was the BEA’s third estimate for the first quarter. Yet at this stage the statisticians have comprehensive data on only 38.5% of GDP. Most of the rest 

In [302]:
docs_folder_dict = get_grouped_docs_dict(retrieved_docs)

In [303]:
for k in list(docs_folder_dict.keys()):
    print(k,len(docs_folder_dict[k] ))

gdp 20


In [304]:
from langchain import PromptTemplate

In [267]:
prompt_template_1artic: str = """You have to answer 5 questions about an article related to the {indicator} Macro-Economic metric number, focusing on the forecast and actual announced number. \n
Following are the questions:\n
Question 1 : Was the sentiment in this article positive? Provide a response as yes or no.\n
Question 2 : Is this number an anomaly or outlier? Provide a response as yes or no.'\n
Question 3: Is this number significant in the context of market dynamics? Provide a response as yes or no.'\n
Question 4: Will this data prompt a re-assessment of the market outlook? Provide a response as yes or no, \n
Question 5 : What are the top 5 key topics in this article? Give a list of key topics separated by commas.\n

{format_instructions}\n

Following is the article text:\n
{article}\n"""

In [268]:
response_schemas_1artic = [
    ResponseSchema(name="Question 1", description="Answer to Question 1 as yes or no"),
    ResponseSchema(name="Question 2", description="Answer to Question 2 as yes or no"),
    ResponseSchema(name="Question 3", description="Answer to Question 3 as yes or no"),
    ResponseSchema(name="Question 4", description="Answer to Question 4 as  yes or no"),
    ResponseSchema(name="Question 5", description="Answer to Question 5 as comma-separated strings")
    
]

output_parser_1artic = StructuredOutputParser.from_response_schemas(response_schemas_1artic)

In [269]:
output_parser_1artic = StructuredOutputParser.from_response_schemas(response_schemas_1artic)

format_instructions_1artic = output_parser_1artic.get_format_instructions()
prompt_1artic = PromptTemplate(
    template=prompt_template_1artic,
    input_variables=["indicator","article"],
    partial_variables={"format_instructions": format_instructions_1artic}
)

In [270]:
len(retrieved_docs)

20

In [271]:
"""
valid = True
for i in range(1,6):
    v = 'Question '+str(i)
    if v not in parsed_dict:
        valid = False
print(valid)
"""

"\nvalid = True\nfor i in range(1,6):\n    v = 'Question '+str(i)\n    if v not in parsed_dict:\n        valid = False\nprint(valid)\n"

In [272]:
"""
i=0
article_text = retrieved_docs[0].page_content
article_id = retrieved_docs[i].metadata['article_id']
folder = retrieved_docs[i].metadata['folder']
textfile = retrieved_docs[i].metadata['textfile']
artic_dateint = getDayMonthYearFromInt( retrieved_docs[i].metadata['date'])
print(artic_dateint)
artic_datetime = datetime(artic_dateint[2],artic_dateint[1],artic_dateint[0])
print(artic_datetime)

tpl_dict = { 'article_id':article_id, 'folder':folder, 'textfile': textfile, 'article_text':article_text }
tpl = ( artic_datetime, tpl_dict  )
print(tpl)
"""

"\ni=0\narticle_text = retrieved_docs[0].page_content\narticle_id = retrieved_docs[i].metadata['article_id']\nfolder = retrieved_docs[i].metadata['folder']\ntextfile = retrieved_docs[i].metadata['textfile']\nartic_dateint = getDayMonthYearFromInt( retrieved_docs[i].metadata['date'])\nprint(artic_dateint)\nartic_datetime = datetime(artic_dateint[2],artic_dateint[1],artic_dateint[0])\nprint(artic_datetime)\n\ntpl_dict = { 'article_id':article_id, 'folder':folder, 'textfile': textfile, 'article_text':article_text }\ntpl = ( artic_datetime, tpl_dict  )\nprint(tpl)\n"

In [202]:
#retrieved_docs[i].metadata

{'article_id': 2466,
 'date': 20210625,
 'folder': 'cpi',
 'textfile': 'cpi_2021-1-1_to_2021-7-1.txt'}

In [247]:
"""
Question 1 : Was the sentiment in this article positive? Respond with Yes, No, or Neutral.\n
Question 2 : Is this number an anomaly or outlier? Reply with 'yes,' 'no,' or 'unclear.'\n
Question 3: Is this number significant in the context of market dynamics? Provide a response of 'yes,' 'no,' or 'not clear.'\n
Question 4: Will this data prompt a re-assessment of the market outlook? Offer a response of 'yes,' 'no,' or 'ambiguous.'\n
Question 5 : What are the top 5 key topics in this article? Give a list of key topics separated by commas.\n
"""

"\nQuestion 1 : Was the sentiment in this article positive? Respond with Yes, No, or Neutral.\n\nQuestion 2 : Is this number an anomaly or outlier? Reply with 'yes,' 'no,' or 'unclear.'\n\nQuestion 3: Is this number significant in the context of market dynamics? Provide a response of 'yes,' 'no,' or 'not clear.'\n\nQuestion 4: Will this data prompt a re-assessment of the market outlook? Offer a response of 'yes,' 'no,' or 'ambiguous.'\n\nQuestion 5 : What are the top 5 key topics in this article? Give a list of key topics separated by commas.\n\n"

In [315]:
result_tpls = []

for i in range(len(retrieved_docs)):
    
    try:
    
        article_text = retrieved_docs[i].page_content
        _input = prompt_1artic.format_prompt(article=article_text,indicator=metric)
        output = model(_input.to_string())
        parsed_dict = output_parser_1artic.parse(output)

        article_text = retrieved_docs[i].page_content
        article_id = retrieved_docs[i].metadata['article_id']
        folder = retrieved_docs[i].metadata['folder']
        textfile = retrieved_docs[i].metadata['textfile']
        artic_dateint = getDayMonthYearFromInt( retrieved_docs[i].metadata['date'])
        artic_datetime = datetime(artic_dateint[2],artic_dateint[1],artic_dateint[0])
        
        datestr = artic_datetime.strftime('%d-%m-%Y')
        
        tpl_dict = { 'article_id':article_id, 'date':datestr, 'folder':folder, 'textfile': textfile}
        
        #for j in range(1,6):
        #    v = 'Question '+str(j)
        #    tpl_dict[v] = parsed_dict[v]
        
        tpl_dict['Positive Sentiment'] = parsed_dict['Question 1']
        tpl_dict['Is Anomaly'] = parsed_dict['Question 2']
        tpl_dict['Is Significant'] = parsed_dict['Question 3']
        tpl_dict['Requires Re-Assessment'] = parsed_dict['Question 4']
        tpl_dict['Keywords'] = parsed_dict['Question 5']
            
        tpl_dict['article_text'] = article_text
        
        tpl = ( artic_datetime, tpl_dict  )
        result_tpls.append(tpl)
        
        print(len(result_tpls))
        
    except:
        pass
    


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [316]:
result_tpls

[(datetime.datetime(2018, 7, 9, 0, 0),
  {'article_id': 13833,
   'date': '09-07-2018',
   'folder': 'gdp',
   'textfile': 'gdp_2018-7-1_to_2019-1-1.txt',
   'Positive Sentiment': 'Yes',
   'Is Anomaly': 'No',
   'Is Significant': 'Yes',
   'Requires Re-Assessment': 'Yes',
   'Keywords': 'GDP, Gross Domestic Income, Measurement Problems, Data Sources, Economic Growth',
   'article_text': 'If you’re looking for the most accurate view of economic growth, you won’t find it in the government’s top-line statistics. Last month the Bureau of Economic Analysis revised its estimate for the first quarter of 2018; the BEA now says gross domestic product grew 2%, annualized, down from the previous figure of 2.2%. This dramatically understates the economy’s actual performance. A better measure, factoring in statistics on incomes, shows growth steaming along at a much stronger 2.8%.\nWhy the difference? In short, because the data aren’t perfect. Unlike the unemployment rate, which comes from a (rela

In [317]:
result_tpls = sorted(result_tpls, key=lambda x: x[0])

In [318]:
for tpl in result_tpls:
    print(tpl[0])

2018-01-09 00:00:00
2018-01-18 00:00:00
2018-01-25 00:00:00
2018-01-25 00:00:00
2018-03-12 00:00:00
2018-03-28 00:00:00
2018-04-23 00:00:00
2018-04-23 00:00:00
2018-04-26 00:00:00
2018-04-26 00:00:00
2018-05-27 00:00:00
2018-07-09 00:00:00
2018-07-09 00:00:00
2018-07-27 00:00:00
2018-07-27 00:00:00
2018-07-27 00:00:00
2018-07-27 00:00:00
2018-11-25 00:00:00
2018-11-29 00:00:00
2018-11-29 00:00:00


In [319]:
result_tpls = [ tpl[1] for tpl in result_tpls ]

In [320]:
for tpl in result_tpls:
    for k in list(tpl.keys()):
        print(k,": ",tpl[k])
    print('_____________')

article_id :  13751
date :  09-01-2018
folder :  gdp
textfile :  gdp_2018-1-1_to_2018-7-1.txt
Positive Sentiment :  Yes
Is Anomaly :  No
Is Significant :  Yes
Requires Re-Assessment :  Yes
Keywords :  Global Economic Prospects, Output Gap, Inflationary Pressures, Monetary Policy, Fiscal Outlook
article_text :  A broad uptick in economic growth around the world in 2017 has led economists at the World Bank to conclude that, for the first time in a decade, the global economy is operating at its potential.
"The year 2018 will likely mark a turning point for the global economy because, for the first time since 2008, the negative global output gap is expected to be closed," the World Bank said in its flagship report on the global economy, known as the Global Economic Prospects report.
At the root of this observation is a calculation that the world economy performed far worse than its potential in the decade after the global financial crisis that sent most of the world's economies into severe

In [277]:
for tpl in result_tpls:
    for k in list(tpl.keys()):
        print(k,": ",tpl[k])
    print('_____________')

article_id :  2475
date :  07-03-2021
folder :  cpi
textfile :  cpi_2021-1-1_to_2021-7-1.txt
Positive Sentiment :  Yes
Is Anomaly :  No
Is Significant :  Yes
Requires Re-Assessment :  Yes
Keywords :  Inflation readings, China’s CPI, US CPI, European Central Bank, US jobless claims, University of Michigan’s consumer sentiment
article_text :  Inflation readings highlight a relatively light week for economic data.
China’s consumer-price index deflation likely persisted in February as food supplies stabilized and costs eased. Economists surveyed by The Wall Street Journal are forecasting CPI contracted 0.4% from a year earlier. Factory-gate prices could be another story. With surging of commodity costs, the country’s producer-price index is forecast to rise 1.4% from the same period last year, compared with a 0.3% increase in January.
Rising gasoline costs are expected to boost overall U.S. consumer prices for February. Core prices, which exclude volatile food and energy components, will l

In [330]:
result_tpls

[{'article_id': 13751,
  'date': '09-01-2018',
  'folder': 'gdp',
  'textfile': 'gdp_2018-1-1_to_2018-7-1.txt',
  'Positive Sentiment': 'Yes',
  'Is Anomaly': 'No',
  'Is Significant': 'Yes',
  'Requires Re-Assessment': 'Yes',
  'Keywords': 'Global Economic Prospects, Output Gap, Inflationary Pressures, Monetary Policy, Fiscal Outlook',
  'article_text': 'A broad uptick in economic growth around the world in 2017 has led economists at the World Bank to conclude that, for the first time in a decade, the global economy is operating at its potential.\n"The year 2018 will likely mark a turning point for the global economy because, for the first time since 2008, the negative global output gap is expected to be closed," the World Bank said in its flagship report on the global economy, known as the Global Economic Prospects report.\nAt the root of this observation is a calculation that the world economy performed far worse than its potential in the decade after the global financial crisis tha

In [328]:
len(result_tpls)

20

In [329]:
type(result_tpls)

list

In [ ]:
sentiments_manual = [1,1,0,0,1]

In [327]:
sentiments

[1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]

In [321]:
sentiments = []
anomalies = []
signifs = []
reassess = []
keywords = []

yndict = {'Yes':1,'No':0}
for tpl in result_tpls:
    sentiments.append(   yndict[tpl['Positive Sentiment']]  )
    anomalies.append( yndict[tpl['Is Anomaly']] )
    signifs.append( yndict[tpl['Is Significant']]  )
    reassess.append( yndict[tpl['Requires Re-Assessment']]  )
    keywords.append( tpl['Keywords']  )

In [290]:
print(sentiments.count(1)/20)
print(anomalies.count(1)/20)
print(signifs.count(1)/20)
print(reassess.count(1)/20)

0.3
0.1
1.0
1.0


In [322]:
print(sentiments.count(1)/20)
print(anomalies.count(1)/20)
print(signifs.count(1)/20)
print(reassess.count(1)/20)

0.55
0.0
1.0
0.95


In [323]:
(0.3+0.55)/2

0.42500000000000004

In [324]:
(0.1+0.0)/2

0.05

In [325]:
(1.0+1.0)/2

1.0

In [326]:
(0.95+1.0)/2

0.975

In [231]:
summaries = []
question1_answers = []

for i in tqdm(range( len(retrieved_docs) )) :
    article_text = retrieved_docs[i].page_content
    _input = prompt_1artic.format_prompt(article=article_text)
    output = model(_input.to_string())
    print(output)
    print('_________')
    parsed_dict = output_parser_1artic.parse(output)
    print(parsed_dict)
    summary = parsed_dict['Summary']
    question1 = parsed_dict['Question 1']
    summaries.append(summary)
    question1_answers.append(question1)
    print('********')

 25%|█████████████████████                                                               | 1/4 [00:03<00:11,  3.75s/it]


```json
{
	"Summary": "U.S. consumer prices rose in December 2020, with gasoline prices accounting for more than 60% of the overall increase. Prices were up 1.4% on a non-seasonally adjusted basis, capping a year that saw muted overall inflation and volatile price swings for some items due to the pandemic. Economists expect the U.S. economic recovery from the pandemic to pick up steam in the second half of this year, with the consumer-price index projected to be 2.4% higher than the same month in 2020. Federal Reserve officials have signaled plans to keep interest rates low as the recovery continues, even if inflation reaches their 2% target.",
	"Question 1": "Neutral"
}
```
_________
{'Summary': 'U.S. consumer prices rose in December 2020, with gasoline prices accounting for more than 60% of the overall increase. Prices were up 1.4% on a non-seasonally adjusted basis, capping a year that saw muted overall inflation and volatile price swings for some items due to the pandemic. Economi

 50%|██████████████████████████████████████████                                          | 2/4 [00:10<00:10,  5.45s/it]


```json
{
	"Summary": "The U.S. consumer prices rose sharply in March as the economic recovery gained momentum, marking the start of an expected monthslong pickup in inflation pressures. The Labor Department reported that its consumer-price index jumped 2.6% in the year ended March, the biggest 12-month increase since August 2018, and rose a seasonally adjusted 0.6% in March from February. Nearly half the monthly increase was due to a 9.1% jump in gasoline prices, which have climbed partly due to production problems following severe winter storms. The core CPI, which excludes the often-volatile categories of food and energy, climbed 1.6% over the prior year, and was up 0.3% in March from February. Economists widely expect consumer prices to keep climbing in the months ahead after nearly a year of muted overall inflation as the Covid-19 pandemic damped consumer spending. The Fed has said it would start to raise interest rates from near zero when PCE inflation reaches 2% and is headed h

 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:16<00:05,  5.74s/it]


```json
{
	"Summary": "The U.S. consumer prices rose sharply in March as the economic recovery gained momentum, marking the start of an expected monthslong pickup in inflation pressures. The Labor Department reported that its consumer-price index jumped 2.6% in the year ended March, the biggest 12-month increase since August 2018, and rose a seasonally adjusted 0.6% in March from February. Nearly half the monthly increase was due to a 9.1% jump in gasoline prices, which have climbed partly due to production problems following severe winter storms. The core CPI, which excludes the often-volatile categories of food and energy, climbed 1.6% over the prior year, and was up 0.3% in March from February. Economists widely expect consumer prices to keep climbing in the months ahead after nearly a year of muted overall inflation as the Covid-19 pandemic damped consumer spending. The Fed has said it would start to raise interest rates from near zero when PCE inflation reaches 2% and is headed h

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.50s/it]


```json
{
	"Summary": "The U.S. consumer prices rose sharply in March as the economic recovery gained momentum, marking the start of an expected monthslong pickup in inflation pressures. The Labor Department reported that its consumer-price index jumped 2.6% in the year ended March, the biggest 12-month increase since August 2018, and rose a seasonally adjusted 0.6% in March from February. Nearly half the monthly increase was due to a 9.1% jump in gasoline prices, which have climbed partly due to production problems following severe winter storms. The core CPI, which excludes the often-volatile categories of food and energy, climbed 1.6% over the prior year, and was up 0.3% in March from February. Economists widely expect consumer prices to keep climbing in the months ahead after nearly a year of muted overall inflation as the Covid-19 pandemic damped consumer spending. The Fed has said it would start to raise interest rates from near zero when PCE inflation reaches 2% and is headed h

In [232]:
len(summaries)

4

In [233]:
question1_answers

['Neutral', 'Neutral', 'Neutral', 'Neutral']

In [234]:
for i in tqdm(range( len(retrieved_docs) )) :
    #article_text = retrieved_docs[i].page_content
    print(retrieved_docs[i])
    print('___________')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<?, ?it/s]

page_content='U.S. consumer prices ticked higher at the end of 2020, as Americans contended with higher gasoline and food costs along with a rise in coronavirus cases across the country.\nThe consumer-price index—which measures what consumers pay for everyday items including food, clothing and recreational activities—increased a seasonally adjusted 0.4% in December compared with November, the Labor Department said Wednesday.\nGasoline prices rose 8.4% over the month, accounting for more than 60% of the overall increase in prices, according to the Labor Department.\nIn the year ended December, prices were up 1.4%, on a non-seasonally adjusted basis, capping a year that saw muted overall inflation and volatile price swings for some items, amid economic disruptions caused by the pandemic. The consumer-price index’s increase during 2020 was the smallest yearly rise since 2015, the Labor Department said.\nState and local governments at the end of last year renewed restrictions on businesses

In [ ]:
#{'article_id': 2478, 'date': 20210413, 'folder': 'cpi', 'textfile': 'cpi_2021-1-1_to_2021-7-1.txt'}
#{'article_id': 83412, 'date': 20210413, 'folder': 'unemployment_rate', 'textfile': 'unemployment-rate_2021-1-1_to_2021-7-1.txt'}

In [ ]:
question_prompt = "What are the contributions of women in economy of USA from 1st September 2018 to 7th November 2019?"

In [243]:
prompt_rag = """ Answer the question given at the end by using the information provided.\n
Following is the information:\n
{article1}\n
{article2}\n
{article3}\n
{article4}\n
Following is the question to give answer to:
{question}\n"""
#prompt_rag_tmplt = PromptTemplate.from_template(template=prompt_rag,input_variables=["article1","article2","article3","article4","question"])
prompt_rag_tmplt = PromptTemplate.from_template(template=prompt_rag)
print(prompt_rag_tmplt)
_input = prompt_rag_tmplt.format_prompt(article1=summaries[0],article2=summaries[1],article3=summaries[2],article4=summaries[3],question=question_prompt)
output = model(_input.to_string())

input_variables=['article1', 'article2', 'article3', 'article4', 'question'] output_parser=None partial_variables={} template=' Answer the question given at the end by using the information provided.\n\nFollowing is the information:\n\n{article1}\n\n{article2}\n\n{article3}\n\n{article4}\n\nFollowing is the question to give answer to:\n{question}\n' template_format='f-string' validate_template=True


In [245]:
_input

StringPromptValue(text=' Answer the question given at the end by using the information provided.\n\nFollowing is the information:\n\nU.S. consumer prices rose in December 2020, with gasoline prices accounting for more than 60% of the overall increase. Prices were up 1.4% on a non-seasonally adjusted basis, capping a year that saw muted overall inflation and volatile price swings for some items due to the pandemic. Economists expect the U.S. economic recovery from the pandemic to pick up steam in the second half of this year, with the consumer-price index projected to be 2.4% higher than the same month in 2020. Federal Reserve officials have signaled plans to keep interest rates low as the recovery continues, even if inflation reaches their 2% target.\n\nThe U.S. consumer prices rose sharply in March as the economic recovery gained momentum, marking the start of an expected monthslong pickup in inflation pressures. The Labor Department reported that its consumer-price index jumped 2.6% 

In [244]:
output

'\nIn 2021, consumer prices in the USA have been rising sharply as the economic recovery gains momentum. The Labor Department reported that the consumer-price index jumped 2.6% in the year ended March, the biggest 12-month increase since August 2018, and rose a seasonally adjusted 0.6% in March from February. Nearly half the monthly increase was due to a 9.1% jump in gasoline prices, which have climbed partly due to production problems following severe winter storms. The core CPI, which excludes the often-volatile categories of food and energy, climbed 1.6% over the prior year, and was up 0.3% in March from February. Economists widely expect consumer prices to keep climbing in the months ahead after nearly a year of muted overall inflation as the Covid-19 pandemic damped consumer spending.'